# 协程

### 迭代器（原有基础上得到新东西）

- 迭代是访问集合的一种方式
- 从集合第一个元素开始访问
- 只向前不后退
- -----
- 迭代器可以节省储存列表的空间，即用即生

In [2]:
from collections import Iterable

# 判断是否可迭代
isinstance([11,22],Iterable)
# isinstance（a，A）
# isinstance用来判断a是否是A的子集

True

```
for循环步骤
1.判断是否可迭代：
    __iter__使其可迭代
2.每次循环调用实例对象的__next__方法：
    __next__用于循环调用
```

In [3]:
# 迭代器原理

from collections import Iterable
from collections import Iterator

class Classmate(object):
    def __init__(self):
        self.names = list()
        
    def add(self, name):
        self.names.append(name)
    
    # 加入__iter__使其可迭代
    # return具有__iter__,__next__方法的引用
    # return的引用就是迭代器
    def __iter__(self):
        
        # 下方self把自己的实例引用传给ClassIterator()的obj
        return ClassIterator(self)

class ClassIterator(object):
    def __init__(self, obj):
        self.obj = obj
        self.num = 0
    
    def __iter__(self):
        pass
    
    def __next__(self):
        if self.num < len(self.obj.names):
            name = self.obj.names[self.num]
            self.num += 1
            return name
        else:
            
            # 停止迭代
            raise StopIteration
    
classmate = Classmate()
classmate.add("1")
classmate.add("2")
classmate.add("3")

# isinstance( ,Iterable)判断是否可迭代
print(isinstance(classmate, Iterable))

# iter()得到__iter__方法返回值，即一个迭代器
iterabor_c = iter(classmate)

# isinstance( ,Iterator)判断是否是迭代器
print(isinstance(classmate, Iterator))

# 调用迭代器的__next__函数
print(next(iterabor_c))

# for循环中，每次循环都调用一次迭代器的__next__函数
for i in classmate:
    print(i)

True
False
1
1
2
3


In [5]:
# 迭代器简化

from collections import Iterable
from collections import Iterator

class Classmate(object):
    def __init__(self):
        self.names = list()
        self.num = 0
        
    def add(self, name):
        self.names.append(name)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.num < len(self.names):
            name = self.names[self.num]
            self.num += 1
            return name
        else:
            raise StopIteration
    
classmate = Classmate()
classmate.add("1")
classmate.add("2")
classmate.add("3")

# for循环中，每次循环都调用一次迭代器的__next__函数
for i in classmate:
    print(i)

1
2
3


```
下述即为迭代器结构：

    def __iter__(self):
        return self
    
    def __next__(self):
        if self.num < len(self.names):
            name = self.names[self.num]
            self.num += 1
            return name
        else:
            raise StopIteration
```

In [17]:
# 斐波纳吉数列

class Fblq(object):
    def __init__(self, n):
        self.l = list()
        self.num = 0
        self.n = n
        self.a = 0
        self.b = 1
        self.f()
        
    def __iter__(self):
        return self
        
    def __next__(self):
        if self.num < self.n:
            ret = self.l[self.num]
            self.num += 1
            return ret
        else:
            raise StopIteration
        
    def f(self):
        for i in range(self.n):
            self.l.append(self.a)
            self.a, self.b = self.b, self.a+self.b
            
fbnq = Fblq(10)

for q in fbnq:
    print(q)
            

0
1
1
2
3
5
8
13
21
34


### 生成器
- 生成器是一种特殊的迭代器
- 边生成边输出，并具有可迭代性

In [4]:
num = [x*2 for x in range(10)]
print(num)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [7]:
# 这是生成器(不常用)，返回一个可迭代对象
nums = (x*2 for x in range(10))
print(nums)
for i in nums:
    print(i)

<generator object <genexpr> at 0x10886fe60>
0
2
4
6
8
10
12
14
16
18


In [25]:
# 生成器

def create_num(numb):
    a, b = 0, 1
    n = 0
    while n < numb:
        # print(a)
        
        # 加入yield使函数成为生成器模版
        # yield返回函数运行到此时a的值
        # yield可以理解为每次迭代输出时，暂停并返回yield
        yield a
        a, b = b, a + b
        n += 1

# 创建可迭代对象
obj = create_num(10)
obj2 = create_num(20)
print(obj)

# next()可以使可迭代对象逐步执行
for i in range(10):
    print(next(obj))

print("-"*30)

while True:
    try:
        r = next(obj2)
        print(r)
    except Exception as ret:
        print(ret.value)
        break

print("-"*30)

# 对obj迭代输出
# next()函数已经对obj迭代输出完毕，所以下述for没有了输出
for c in obj:
    print(c)

<generator object create_num at 0x108c627d8>
0
1
1
2
3
5
8
13
21
34
------------------------------
0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
None
------------------------------


- 生成器创建的对象互不干涉，不影响next()输出

### send激活
- 除next()可以激活外，send也可以激活

In [32]:
def create_nums(numb):
    a, b = 0, 1
    n = 0
    while n < numb:
        r = yield a  # 激活时“暂停”
        print(">>>>>>",r)
        a, b = b, a + b
        n += 1
        
o = create_nums(10)

# send无法放在生成器开始时
# 生成器开始时因为还没有“暂停”，所以无法替换
ret = next(o)
# 下述方法在生成器开始时使用也可以
# ret = o.send(None)

print(ret)

# next()后生成器暂停，此时可以使用send()发送给"yield a"
# send()括号中的参数会传给生成器中的"yield a"，并替换
ret = o.send("liduo")
print(ret)

0
>>>>>> liduo
1


In [37]:
o = create_nums(10)
next(o)
while True:
    try:
        r = o.send("liduo")
        print(r)
    except Exception as ret:
        break

>>>>>> liduo
1
>>>>>> liduo
1
>>>>>> liduo
2
>>>>>> liduo
3
>>>>>> liduo
5
>>>>>> liduo
8
>>>>>> liduo
13
>>>>>> liduo
21
>>>>>> liduo
34
>>>>>> liduo


# 多任务(yield)

In [39]:
import time

def test1():
    while True:
        print("---1---")
        time.sleep(0.5)
        yield   # 用于暂停
        
def test2():
    while True:
        print("---2---")
        time.sleep(0.5)
        yield   # 用于暂停
        
def main():
    t1 = test1()
    t2 = test2()
    i = 0
    while i < 10:
        next(t1) # 触发暂停
        next(t2) # 触发暂停
        i += 1
        
if __name__ == "__main__":
    main()

---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---


# 多任务(greenlet)

In [6]:
from greenlet import greenlet 
import time

def test_1():
    for i in range(5):
        print("---1---")
        g2.switch()
        time.sleep(0.2)
        
def test_2():
    for i in range(5):
        print("---2---")
        g1.switch()
        time.sleep(0.2)
        
g1 = greenlet(test_1)
g2 = greenlet(test_2)

g1.switch()

---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---
---1---
---2---


# 多任务(gevent)
- 有延时才切换

In [4]:
# 无延时，不切换任务
import gevent

def f(n):
    for i in range(n):
        print(gevent.getcurrent(),i)
        
g1 = gevent.spawn(f, 5)
g2 = gevent.spawn(f, 4)
g3 = gevent.spawn(f, 3)

# 开始运行
g1.join()
g2.join()
g3.join()

<Greenlet at 0x1091b1508: f(5)> 0
<Greenlet at 0x1091b1508: f(5)> 1
<Greenlet at 0x1091b1508: f(5)> 2
<Greenlet at 0x1091b1508: f(5)> 3
<Greenlet at 0x1091b1508: f(5)> 4
<Greenlet at 0x1091b15a0: f(4)> 0
<Greenlet at 0x1091b15a0: f(4)> 1
<Greenlet at 0x1091b15a0: f(4)> 2
<Greenlet at 0x1091b15a0: f(4)> 3
<Greenlet at 0x1091b10e0: f(3)> 0
<Greenlet at 0x1091b10e0: f(3)> 1
<Greenlet at 0x1091b10e0: f(3)> 2


In [7]:
# 延时，切换任务
import gevent

def f(n):
    for i in range(n):
        print(gevent.getcurrent(),i)
        # 遇到延时，切换执行
        gevent.sleep(0.1)
        
g1 = gevent.spawn(f, 5)
g2 = gevent.spawn(f, 4)
g3 = gevent.spawn(f, 3)

# 遇到阻塞，开始执行
g1.join()
g2.join()
g3.join()

<Greenlet at 0x1091b1470: f(5)> 0
<Greenlet at 0x1091b1178: f(4)> 0
<Greenlet at 0x1091b12a8: f(3)> 0
<Greenlet at 0x1091b1470: f(5)> 1
<Greenlet at 0x1091b1178: f(4)> 1
<Greenlet at 0x1091b12a8: f(3)> 1
<Greenlet at 0x1091b1470: f(5)> 2
<Greenlet at 0x1091b1178: f(4)> 2
<Greenlet at 0x1091b12a8: f(3)> 2
<Greenlet at 0x1091b1470: f(5)> 3
<Greenlet at 0x1091b1178: f(4)> 3
<Greenlet at 0x1091b1470: f(5)> 4


In [10]:
# 延时操作使用原生
import time
import gevent
# from gevent import monkey

# 加入下述代码，实现使用原生延时操作
gevent.monkey.patch_all()

def f(n):
    for i in range(n):
        print(gevent.getcurrent(),i)
        # 遇到延时，切换执行
        time.sleep(0.1)
        
g1 = gevent.spawn(f, 5)
g2 = gevent.spawn(f, 4)
g3 = gevent.spawn(f, 3)

# 遇到阻塞，开始执行
g1.join()
g2.join()
g3.join()

<Greenlet at 0x108d129c8: f(5)> 0
<Greenlet at 0x108d12a60: f(4)> 0
<Greenlet at 0x108d12af8: f(3)> 0
<Greenlet at 0x108d129c8: f(5)> 1
<Greenlet at 0x108d12a60: f(4)> 1
<Greenlet at 0x108d12af8: f(3)> 1
<Greenlet at 0x108d129c8: f(5)> 2
<Greenlet at 0x108d12a60: f(4)> 2
<Greenlet at 0x108d12af8: f(3)> 2
<Greenlet at 0x108d129c8: f(5)> 3
<Greenlet at 0x108d12a60: f(4)> 3
<Greenlet at 0x108d129c8: f(5)> 4


In [ ]:
# 使用joinall()减少代码量
import time
import gevent
from gevent import monkey

monkey.patch_all()

def f(n):
    for i in range(n):
        print(gevent.getcurrent(),i)
        # 遇到延时，切换执行
        time.sleep(0.1)
gevent.joinall([gevent.spawn(f, 5),
                gevent.spawn(f, 4),
                gevent.spawn(f, 3)])

<Greenlet at 0x108964800: f(5)> 0
<Greenlet at 0x108964930: f(4)> 0
<Greenlet at 0x1089649c8: f(3)> 0
<Greenlet at 0x108964800: f(5)> 1
<Greenlet at 0x108964930: f(4)> 1
<Greenlet at 0x1089649c8: f(3)> 1
<Greenlet at 0x108964800: f(5)> 2
<Greenlet at 0x108964930: f(4)> 2
<Greenlet at 0x1089649c8: f(3)> 2
<Greenlet at 0x108964800: f(5)> 3
<Greenlet at 0x108964930: f(4)> 3
<Greenlet at 0x108964800: f(5)> 4
